In [3]:
import numpy as np
import pandas as pd

In [96]:
datasets = [
    "mendelian_matched_9",
    "gwas_matched_9",
]

subsets = [
    "all",
    "missense_variant",
    "non_missense",
]

modalities = [
    "Zero-shot",
    "Linear probing",
]

models = [
    "Ensemble",
    "CADD",
    "GPN-MSA",
    "NucleotideTransformer",
    "HyenaDNA",
    "Caduceus",
    "GPN",
    "Enformer",
    "Borzoi",
]

def get_model_path(model, modality, dataset, subset):
    supervised_suffix = "LogisticRegression.chrom"
    if model == "CADD":
        predictor = "CADD.plus.RawScore" if modality == "Zero-shot" else f"CADD.{supervised_suffix}"
    elif model in ["Enformer", "Borzoi"]:
        predictor = f"{model}_L2_L2.plus.all" if modality == "Zero-shot" else f"{model}.{supervised_suffix}"
    elif model == "Ensemble":
        if "mendelian" in dataset:
            prefix = "OMIM_Ensemble_v2" 
        else:
            prefix = "Enformer+GPN-MSA+CADD"
        predictor = f"{prefix}.{supervised_suffix}"
    else:
        if "mendelian" in dataset:
            llr_version = "LLR"
            sign = "minus"
        elif "gwas" in dataset:
            llr_version = "absLLR"
            sign = "plus"
        predictor = f"{model}_{llr_version}.{sign}.score" if modality == "Zero-shot" else f"{model}_{llr_version}+InnerProducts.{supervised_suffix}"
    return f"../../results/dataset/{dataset}/metrics/{subset}/{predictor}.csv"

In [116]:
rows = []
for dataset in datasets:
    for subset in subsets:
        for modality in modalities:
            for model in models:
                if model == "Ensemble" and modality == "Zero-shot":
                    continue
                path = get_model_path(model, modality, dataset, subset)
                df = pd.read_csv(path).iloc[0]
                rows.append([dataset, subset, modality, model, df["score"], df["se"]])
df = pd.DataFrame(rows, columns=["dataset", "subset", "modality", "model", "score", "se"])
df

,dataset,subset,modality,model,score,se
0,mendelian_matched_9,all,Zero-shot,CADD,0.696187,0.031779
1,mendelian_matched_9,all,Zero-shot,GPN-MSA,0.728062,0.025896
2,mendelian_matched_9,all,Zero-shot,NucleotideTransformer,0.110495,0.011018
3,mendelian_matched_9,all,Zero-shot,HyenaDNA,0.095783,0.001784
4,mendelian_matched_9,all,Zero-shot,Caduceus,0.091912,0.001990
...,...,...,...,...,...,...
97,gwas_matched_9,non_missense,Linear probing,HyenaDNA,0.114883,0.007000
98,gwas_matched_9,non_missense,Linear probing,Caduceus,0.137891,0.008115
99,gwas_matched_9,non_missense,Linear probing,GPN,0.170151,0.006661
100,gwas_matched_9,non_missense,Linear probing,Enformer,0.279787,0.009327


In [117]:
def format_score(x):
    return (x * 100).round().astype(int).apply(lambda y: f"{y:02d}")

def format_se(x):
    assert x.max() < 10
    return (x * 100).round().astype(int).apply(lambda y: f"{y:01d}")

df["value"] = format_score(df.score) + "$\pm$" + format_se(df.se)
#df["value"] = df.score.apply(lambda x: f"{x:.2f}") + "$\pm$" + df.se.apply(lambda x: f"{x:.2f}")

In [118]:
df.loc[(df.model.isin(["Enformer", "Borzoi", "GPN"])) & (df.subset!="non_missense"), "value"] = "-"

In [119]:
bold_values = [
    ("mendelian_matched_9", "all", "Zero-shot", "CADD"),
    ("mendelian_matched_9", "all", "Zero-shot", "GPN-MSA"),
    ("mendelian_matched_9", "missense_variant", "Zero-shot", "CADD"),
    ("mendelian_matched_9", "missense_variant", "Zero-shot", "GPN-MSA"),
    ("mendelian_matched_9", "non_missense", "Zero-shot", "CADD"),
    ("mendelian_matched_9", "non_missense", "Zero-shot", "GPN-MSA"),

    ("mendelian_matched_9", "all", "Linear probing", "CADD"),
    ("mendelian_matched_9", "all", "Linear probing", "Ensemble"),
    ("mendelian_matched_9", "missense_variant", "Linear probing", "CADD"),
    ("mendelian_matched_9", "missense_variant", "Linear probing", "Ensemble"),
    ("mendelian_matched_9", "non_missense", "Linear probing", "CADD"),
    ("mendelian_matched_9", "non_missense", "Linear probing", "Ensemble"),

    ("gwas_matched_9", "all", "Zero-shot", "CADD"),
    ("gwas_matched_9", "all", "Zero-shot", "GPN-MSA"),
    ("gwas_matched_9", "missense_variant", "Zero-shot", "CADD"),
    #("gwas_matched_9", "missense_variant", "Zero-shot", "GPN-MSA"),
    ("gwas_matched_9", "non_missense", "Zero-shot", "Enformer"),
    ("gwas_matched_9", "non_missense", "Zero-shot", "Borzoi"),

    ("gwas_matched_9", "all", "Linear probing", "Ensemble"),
    ("gwas_matched_9", "missense_variant", "Linear probing", "Ensemble"),
    ("gwas_matched_9", "non_missense", "Linear probing", "Ensemble"),
]

for dataset, subset, modality, model in bold_values:
    mask = (
        (df.dataset==dataset) & (df.subset==subset) &
        (df.modality==modality) & (df.model==model)
    )
    df.loc[mask, "value"] = r"\textbf{" + df.loc[mask, "value"] + "}"

In [120]:
df.dataset = df.dataset.map({
    "mendelian_matched_9": r"\textbf{Mendelian traits}",
    "gwas_matched_9": r"\textbf{Complex traits}",
})
df.subset = df.subset.map({
    "all": "All",
    "missense_variant": "Coding",
    "non_missense": "Non-coding",
})
df.modality = df.modality.map({
    "Zero-shot": r"\textbf{Zero-shot}",
    "Linear probing": r"\textbf{Linear probing}",
})

In [121]:
df = df.pivot_table(
    index=["modality", "model"], columns=["dataset", "subset"], values="value",
    aggfunc="first", sort=False,
)
df

dataset                                       \textbf{Mendelian traits}  \
subset                                                              All   
modality                model                                             
\textbf{Zero-shot}      CADD                          \textbf{70$\pm$3}   
                        GPN-MSA                       \textbf{73$\pm$3}   
                        NucleotideTransformer                  11$\pm$1   
                        HyenaDNA                               10$\pm$0   
                        Caduceus                               09$\pm$0   
                        GPN                                           -   
                        Enformer                                      -   
                        Borzoi                                        -   
\textbf{Linear probing} CADD                          \textbf{91$\pm$2}   
                        GPN-MSA                                74$\pm$3   
                        NucleotideTransformer                  15$\pm$1   
                        HyenaDNA                               13$\pm$1   
                        Caduceus                               11$\pm$1   
                        GPN                                           -   
                        Enformer                                      -   
                        Borzoi                                        -   
                        Ensemble                      \textbf{92$\pm$1}   

dataset                                                           \
subset                                                    Coding   
modality                model                                      
\textbf{Zero-shot}      CADD                   \textbf{85$\pm$2}   
                        GPN-MSA                \textbf{82$\pm$2}   
                        NucleotideTransformer           12$\pm$2   
                        HyenaDNA                        09$\pm$0   
                        Caduceus                        09$\pm$0   
                        GPN                                    -   
                        Enformer                               -   
                        Borzoi                                 -   
\textbf{Linear probing} CADD                   \textbf{92$\pm$2}   
                        GPN-MSA                         81$\pm$3   
                        NucleotideTransformer           17$\pm$1   
                        HyenaDNA                        12$\pm$2   
                        Caduceus                        11$\pm$1   
                        GPN                                    -   
                        Enformer                               -   
                        Borzoi                                 -   
                        Ensemble               \textbf{93$\pm$2}   

dataset                                                           \
subset                                                Non-coding   
modality                model                                      
\textbf{Zero-shot}      CADD                   \textbf{68$\pm$4}   
                        GPN-MSA                \textbf{67$\pm$4}   
                        NucleotideTransformer           10$\pm$0   
                        HyenaDNA                        10$\pm$1   
                        Caduceus                        09$\pm$0   
                        GPN                             33$\pm$7   
                        Enformer                        37$\pm$4   
                        Borzoi                          39$\pm$5   
\textbf{Linear probing} CADD                   \textbf{82$\pm$3}   
                        GPN-MSA                         65$\pm$6   
                        NucleotideTransformer           12$\pm$2   
                        HyenaDNA                        12$\pm$1   
                        Caduceus                        13$\pm$3   
                        GPN                       

In [122]:
df.index.names = [None, None]
df.columns.names = [None, None]

In [123]:
print(df.to_latex(multicolumn_format='c', escape=False))

\begin{tabular}{llllllll}
\toprule
 &  & \multicolumn{3}{c}{\textbf{Mendelian traits}} & \multicolumn{3}{c}{\textbf{Complex traits}} \\
 &  & All & Coding & Non-coding & All & Coding & Non-coding \\
\midrule
\multirow[t]{8}{*}{\textbf{Zero-shot}} & CADD & \textbf{70$\pm$3} & \textbf{85$\pm$2} & \textbf{68$\pm$4} & \textbf{16$\pm$0} & \textbf{23$\pm$2} & 19$\pm$1 \\
 & GPN-MSA & \textbf{73$\pm$3} & \textbf{82$\pm$2} & \textbf{67$\pm$4} & \textbf{16$\pm$1} & \textbf{19$\pm$2} & 18$\pm$1 \\
 & NucleotideTransformer & 11$\pm$1 & 12$\pm$2 & 10$\pm$0 & 09$\pm$0 & 09$\pm$0 & 09$\pm$0 \\
 & HyenaDNA & 10$\pm$0 & 09$\pm$0 & 10$\pm$1 & 10$\pm$0 & 09$\pm$0 & 10$\pm$0 \\
 & Caduceus & 09$\pm$0 & 09$\pm$0 & 09$\pm$0 & 09$\pm$0 & 09$\pm$0 & 09$\pm$0 \\
 & GPN & - & - & 33$\pm$7 & - & - & 10$\pm$0 \\
 & Enformer & - & - & 37$\pm$4 & - & - & \textbf{22$\pm$1} \\
 & Borzoi & - & - & 39$\pm$5 & - & - & \textbf{21$\pm$1} \\
\cline{1-8}
\multirow[t]{9}{*}{\textbf{Linear probing}} & CADD & \textbf{91$\pm$2